## Modelo datos estructurados
Este notebook desarrolla un primer modelo para resolver el problema de Petfinder. Empezamos haciendo un modelo inicial muy simple para ver la viabilidad de resolver el problema. Luego analizamos como se comporta la métrica kappa propuesta y vemos la matriz de confusión. Finalmente hacemos una optimizacin de hiperparametros evaluando con train/test y otra validando con 5 fold CV y testeando en el 20% de los datos

In [37]:
import os
os.chdir('/Users/ri1965/Desktop/Maestria DS Austral 2024/11 - Labo 02/work/')
print("Directorio actual:", os.getcwd())

Directorio actual: /Users/ri1965/Desktop/Maestria DS Austral 2024/11 - Labo 02/work


In [38]:
import os
print("Directorio actual:", os.getcwd())
print("Archivos aquí:", os.listdir())

Directorio actual: /Users/ri1965/Desktop/Maestria DS Austral 2024/11 - Labo 02/work
Archivos aquí: ['optuna_artifacts', 'eda', '.DS_Store', 'optuna_temp_artifacts', 'LGBM_prueba.ipynb', 'db.sqlite3', 'LGBM_base.ipynb', 'models', '__pycache__', '04_Tabulares.ipynb', 'utiles.py', 'LGBM_prueba.pdf', '.ipynb_checkpoints', 'LGBM_fe.ipynb']


In [39]:
#Import de librerias basicas tablas y matrices
import numpy as np 
import pandas as pd 

#Gradient Boosting
import lightgbm as lgb

#Funciones auxiliares sklearn
from sklearn.model_selection import train_test_split, StratifiedKFold #Split y cross Validation
from sklearn.metrics import cohen_kappa_score, accuracy_score, balanced_accuracy_score #Metricas
from sklearn.utils import shuffle 

#Visualizacióon
from plotly import express as px

#Plot de matriz de confusion normalizada en actuals
from utiles import plot_confusion_matrix

import os

#Optimizacion de hiperparametros
import optuna
from optuna.artifacts import FileSystemArtifactStore, upload_artifact

#Guardado de objetos en archivos joblib
from joblib import load, dump


In [40]:
# Paths para acceso archivos
#Este notebook asume la siguiente estructura de carpetas a partir de la ubicacion de base_dir 
#(dos niveles arriba de la cƒarpeta donde se ejecuta el notebook). 
# /ƒ/ƒ
# /UA_MDM_Labo2/inputƒ
# /UA_MDM_Labo2/input/petfinder-adoption-prediction/            <- Aca deben ir todos los archivos de datos de la competencia 
# /UA_MDM_Labo2/tutoriales/                       <- Aca deben poner los notebooks y scripts que les compartimos
# /UA_MDM_Labo2/work/                             <- Resultados de notebooks iran dentro de esta carpeta en subcarpetas
# /UA_MDM_Labo2/work/models/                     <- Modelos entrenados en archivos joblibs
# /UA_MDM_Labo2/work/optuna_temp_artifacts/      <- Archivos que queremos dejar como artefacto de un trial de optuna (optuna los copiara a la carpeta de abajo)
# /UA_MDM_Labo2/work/optuna_artifacts/           <- Archivos con artefactos que sibimos a optuna

#Subimos dos niveles para quedar en la carpeta que contiene input y UA_MDM_Labo2
BASE_DIR = '../'

#Datos de entrenamiento 
PATH_TO_TRAIN = os.path.join(BASE_DIR, "input/petfinder-adoption-prediction/train/train_preprocessed.csv")

#Salida de modelos entrenados
PATH_TO_MODELS = os.path.join(BASE_DIR, "work/models")

#Artefactos a subir a optuna
PATH_TO_TEMP_FILES = os.path.join(BASE_DIR, "work/optuna_temp_artifacts")

#Artefactos que optuna gestiona
PATH_TO_OPTUNA_ARTIFACTS = os.path.join(BASE_DIR, "work/optuna_artifacts")


SEED = 42 #Semilla de procesos aleatorios (para poder replicar exactamente al volver a correr un modelo)
TEST_SIZE = 0.2 #Facción para train/test= split

In [41]:
# Datos Tabulares
dataset = pd.read_csv(PATH_TO_TRAIN)

In [42]:
#Columnas del dataset
dataset.columns

Index(['num__Type', 'num__Age', 'num__Breed1', 'num__Breed2', 'num__Gender',
       'num__Color1', 'num__Color2', 'num__Color3', 'num__MaturitySize',
       'num__FurLength', 'num__Vaccinated', 'num__Dewormed', 'num__Sterilized',
       'num__Health', 'num__Quantity', 'num__Fee', 'num__State',
       'num__VideoAmt', 'num__PhotoAmt', 'num__AdoptionSpeed', 'cat__Name',
       'cat__RescuerID', 'cat__Description', 'cat__PetID', 'AdoptionSpeed'],
      dtype='object')

In [43]:
#Separo un 20% para test estratificado opr target
train, test = train_test_split(dataset,
                               test_size = TEST_SIZE,
                               random_state = SEED,
                               stratify = dataset.AdoptionSpeed)

In [44]:
#Armo listas con features de texto y numericas
char_feats = [f for f in dataset.columns if dataset[f].dtype=='O']
numeric_feats = [f for f in dataset.columns if dataset[f].dtype!='O']

In [45]:
#Lista de features numericas
numeric_feats

['num__Type',
 'num__Age',
 'num__Breed1',
 'num__Breed2',
 'num__Gender',
 'num__Color1',
 'num__Color2',
 'num__Color3',
 'num__MaturitySize',
 'num__FurLength',
 'num__Vaccinated',
 'num__Dewormed',
 'num__Sterilized',
 'num__Health',
 'num__Quantity',
 'num__Fee',
 'num__State',
 'num__VideoAmt',
 'num__PhotoAmt',
 'num__AdoptionSpeed',
 'cat__Name',
 'cat__RescuerID',
 'cat__Description',
 'cat__PetID',
 'AdoptionSpeed']

In [46]:

#Defino features a usar en un primer modelo de prueba
features = ['Type',
 'Age',
 'Breed1',
 'Breed2',
 'Gender',
 'Color1',
 'Color2',
 'Color3',
 'MaturitySize',
 'FurLength',
 'Vaccinated',
 'Dewormed',
 'Sterilized',
 'Health',
 'Quantity',
 'Fee',
 'State',
 'VideoAmt',
 'PhotoAmt']

label = 'AdoptionSpeed'

In [47]:
# Nombre de la variable objetivo
label = 'AdoptionSpeed'

# Conjunto de predictores: todas las columnas menos la target
features = train.columns.drop(label)

# --- DataFrames de entrenamiento ---
X_train = train[features].copy()
y_train = train[label].copy()

# --- DataFrames de prueba ---
X_test  = test[features].copy()
y_test  = test[label].copy()

In [48]:
X_train = X_train.drop('num__AdoptionSpeed', axis=1)
X_test = X_test.drop('num__AdoptionSpeed', axis=1)


In [49]:
y_test

14696    4
14823    4
2838     4
1848     2
669      4
        ..
996      4
12222    1
10538    1
11062    4
4858     4
Name: AdoptionSpeed, Length: 2999, dtype: int64

In [50]:
#Entreno un modelo inicial sin modificar hiperparametros. Solamente especifico el numero de clases y el tipo de modelo como clasificacoión
lgb_params = params = {
                        'objective': 'multiclass',
                        'num_class': len(y_train.unique())
                        }


#genero el objeto Dataset que debo pasarle a lightgbm para que entrene
lgb_train_dataset = lgb.Dataset(data=X_train,
                                label=y_train)

#entreno el modelo con los parametros por defecto
lgb_model = lgb.train(lgb_params,
                      lgb_train_dataset)

In [51]:
lgb_model.predict(X_test).argmax(axis=1)

array([3, 4, 4, ..., 1, 2, 4])

In [52]:
#Obtengo las predicciones sobre el set de test. El modelo me da una lista de probabilidades para cada clase y tomo la clase con mayor probabilidad con la funcion argmax
y_pred = lgb_model.predict(X_test).argmax(axis=1)

#Calculo el Kappa
cohen_kappa_score(y_test,y_pred, weights = 'quadratic')

0.33233487606532697

In [53]:
#Muestro la matriz de confusión
display(plot_confusion_matrix(y_test,y_pred))

In [54]:
#Vamos a ponewr en perspectiva el score de Kappa


#Cual es el score perfecto? Evaluo la clase real contra si misma. Es decir, el caso en que el modelo establece todas las clases en su valor real
cohen_kappa_score(y_test,y_test, weights = 'quadratic')

1.0

In [55]:
#Como se veria la matriz de confusión
display(plot_confusion_matrix(y_test,y_test))

In [56]:
#Vamos a ver como se comporta kappa si hafgo una predicción al azar (respetando las proporciones de cada clase)
y_shuffled = shuffle(y_test,
                     random_state = 42)


#Genero diccionarios para cambiar algunas predicicones reales por una prediccion cercana y_cerca y una lejana y_lejos a la real 
# ejemplo: la real se 0 voy a estimar 1 para la cercana y 4 para la lejana
dict_map_cerca = {0:1,
                  1:2,
                  2:3,
                  3:4,
                  4:3}

dict_map_lejos = {0:4,
                  1:4,
                  2:0,
                  3:0,
                  4:0}

y_cerca = [dict_map_cerca[i] for i in y_test]

y_lejos = [dict_map_lejos[i] for i in y_test]


In [57]:

#Vamos a simular que la probabilidad de tener la prediccion real en casa muestra varia de 0 a 100. 
#Genero una numero aleatorio para cada muestra
random_list =  np.random.rand(len(y_test))

#inicializo un dataframe de resultados vacio
kappa_progression = pd.DataFrame()

#La variable i tiene un umbral para ir variando la cantidad de aciertos desde 0% a 100%
for i in range(101):

    #Genero la prediccion para i% de aciertos donde cuando no acierto me quedo con una prediccion al azar (podria ser la "correcta" pero solo por azar)
    y_simulado = [y_test.iloc[sample] if random_list[sample]<i/100 else y_shuffled.iloc[sample] for sample in range(len(y_test))]

    #Genero la prediccion para i% de aciertos donde cuando no acierto me quedo con una prediccion cercana o lejana a la correcta
    y_simulado_cerca = [y_test.iloc[sample] if random_list[sample]<i/100 else y_cerca[sample] for sample in range(len(y_test))]
    y_simulado_lejos = [y_test.iloc[sample] if random_list[sample]<i/100 else y_lejos[sample] for sample in range(len(y_test))]


    #Grabo los resultados en un dataframe para cada i% de aciertos
    kappa_progression = pd.concat([kappa_progression,
                                   pd.DataFrame({'Conocidos':[i],
                                                'kappa':cohen_kappa_score(y_test,
                                                                        y_simulado,
                                                                        weights = 'quadratic'),
                                                'kappa_cerca':cohen_kappa_score(y_test,
                                                                        y_simulado_cerca,
                                                                        weights = 'quadratic'),
                                                'kappa_lejos':cohen_kappa_score(y_test,
                                                                        y_simulado_lejos,
                                                                        weights = 'quadratic'),                                                                        
                                                'accuracy':accuracy_score(y_test,
                                                                        y_simulado),
                                                'balanced_accuracy':balanced_accuracy_score(y_test,
                                                                        y_simulado),
                                                                        })],
                ignore_index=True)

In [58]:
#Grafico el comportamiento de la métrica a medida que incremento los aciertos. Tambien muestro lor resultados de otras metricas como Accuracy y Balanced Accuracy
px.line(kappa_progression,x='Conocidos',y=['kappa',
                                           'kappa_cerca',
                                           'kappa_lejos',
                                           'accuracy',
                                           'balanced_accuracy'])

In [59]:

#A modo de ejemplo muestro kappa y matriz de confusion para 50% de aciertos donde los errores quedan cerca de la clase correcta
y_simulado_cerca = [y_test.iloc[sample] if random_list[sample]<50/100 else y_cerca[sample] for sample in range(len(y_test))]

display(plot_confusion_matrix(y_test,y_simulado_cerca, 
                              title = "Kappa " + str(cohen_kappa_score(y_test,y_simulado_cerca, weights = 'quadratic'))))



In [60]:

#A modo de ejemplo muestro kappa y matriz de confusion para 50% de aciertos donde los errores quedan lejos de la clase correcta
y_simulado_lejos = [y_test.iloc[sample] if random_list[sample]<50/100 else y_lejos[sample] for sample in range(len(y_test))]

display(plot_confusion_matrix(y_test,y_simulado_lejos, 
                              title = "Kappa " + str(cohen_kappa_score(y_test,y_simulado_lejos, weights = 'quadratic'))))


In [61]:
#Pruebo un modelo alternativo donde en vez de usar la version multiclass real de lightGBM utilizo One vs All

lgb_params = params = {
                        'objective': 'multiclassova',
                        'num_class': len(y_train.unique())
                        }


lgb_train_dataset = lgb.Dataset(data=X_train,
                                label=y_train)


lgb_model = lgb.train(lgb_params,
                      lgb_train_dataset)

In [62]:
#MAtriz de confusion y Kappa dfe OVA
y_pred = lgb_model.predict(X_test).argmax(axis=1)

display(plot_confusion_matrix(y_test,y_pred))

{'kappa':cohen_kappa_score(y_test,
                y_pred,
                weights = 'quadratic'),
 'accuracy':accuracy_score(y_test,y_pred),
 'balanced_accuracy':balanced_accuracy_score(y_test,y_pred)}




{'kappa': 0.3219611858754706,
 'accuracy': 0.3934644881627209,
 'balanced_accuracy': 0.32226243024503964}

## Optimizacion de hiperparametros modelo train/test

In [63]:

#Funcion que vamos a optimizar. Optuna requiere que usemos el objeto trial para generar los parametros a optimizar
def lgb_objective(trial):
    #PArametros para LightGBM
    lgb_params = {      
                        #PArametros fijos
                        'objective': 'multiclass',
                        'verbosity':-1,
                        'num_class': len(y_train.unique()),
                        #Hiperparametros a optimizar utilizando suggest_float o suggest_int segun el tipo de dato
                        #Se indica el nombre del parametro, valor minimo, valor maximo 
                        #en elgunos casos el parametro log=True para parametros que requieren buscar en esa escala
                        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
                        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),
                        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
                        'feature_fraction': trial.suggest_float('feature_fraction', 0.4, 1.0),
                        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.4, 1.0),
                        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
                        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
                        } 

    #Genero objeto dataset de entrenamiento
    lgb_train_dataset = lgb.Dataset(data=X_train,
                                    label=y_train)

    #ajuste de modelo
    lgb_model = lgb.train(lgb_params,
                        lgb_train_dataset)
    
    #Devuelvo el score en test
    return(cohen_kappa_score(y_test,lgb_model.predict(X_test).argmax(axis=1),
                             weights = 'quadratic'))

In [64]:
#Defino el estudio a optimizar
study = optuna.create_study(direction='maximize', #buscamos maximizar la metrica
                            storage="sqlite:///../work/db.sqlite3",  # Specify the storage URL here.
                            study_name="05 - LGB Multiclass Preproc", #nombre del experimento
                            load_if_exists=True) #continuar si ya existe

#Corremos 100 trials para buscar mejores parametros
study.optimize(lgb_objective, n_trials=10)

[I 2025-04-15 12:36:40,092] Using an existing study with name '05 - LGB Multiclass Preproc' instead of creating a new one.
[I 2025-04-15 12:36:41,971] Trial 10 finished with value: 0.3104356710941265 and parameters: {'lambda_l1': 0.004129392074519968, 'lambda_l2': 8.979997100841587, 'num_leaves': 237, 'feature_fraction': 0.4194640912886127, 'bagging_fraction': 0.5742153008328528, 'bagging_freq': 3, 'min_child_samples': 72}. Best is trial 0 with value: 1.0.
[I 2025-04-15 12:36:43,288] Trial 11 finished with value: 0.3189780730537808 and parameters: {'lambda_l1': 0.00011172338780377144, 'lambda_l2': 0.04302645152989335, 'num_leaves': 256, 'feature_fraction': 0.9935190546329147, 'bagging_fraction': 0.5320560815569592, 'bagging_freq': 3, 'min_child_samples': 77}. Best is trial 0 with value: 1.0.
[I 2025-04-15 12:36:44,201] Trial 12 finished with value: 0.32177146207124585 and parameters: {'lambda_l1': 2.6611772975390763e-05, 'lambda_l2': 1.2356567468328212e-07, 'num_leaves': 204, 'feature_

In [65]:
#Obtenemos mejor resultado
study.best_params

{'lambda_l1': 6.331102291634836e-07,
 'lambda_l2': 2.746005902235411,
 'num_leaves': 253,
 'feature_fraction': 0.6616339602835859,
 'bagging_fraction': 0.6245923579961066,
 'bagging_freq': 3,
 'min_child_samples': 70}

In [66]:
#Vamos a replicar el resultado de la optimizacion reentrenando el modelo con el mejor conjunto de hiperparametros
#Generamos parametros incluyendo los fijos y la mejor solución que encontro optuna
lgb_params =  {      
                        'objective': 'multiclass',
                        'verbosity':-1,
                        'num_class': len(y_train.unique())} | study.best_params

lgb_train_dataset = lgb.Dataset(data=X_train,
                                label=y_train)


#Entreno
lgb_model = lgb.train(lgb_params,
                    lgb_train_dataset)

#Muestro matriz de confusion y kappa
display(plot_confusion_matrix(y_test,lgb_model.predict(X_test).argmax(axis=1)))

cohen_kappa_score(y_test,lgb_model.predict(X_test).argmax(axis=1),
                             weights = 'quadratic')


0.3140165915656826

## Modelo con cross validation y conjunto de test

In [67]:
#Genero una metrica para que lightGBM haga la evaluación y pueda hacer early_stopping en el cross validation
def lgb_custom_metric_kappa(dy_pred, dy_true):
    metric_name = 'kappa'
    value = cohen_kappa_score(dy_true.get_label(),dy_pred.argmax(axis=1),weights = 'quadratic')
    is_higher_better = True
    return(metric_name, value, is_higher_better)

#Funcion objetivo a optimizar. En este caso vamos a hacer 5fold cv sobre el conjunto de train. 
# El score de CV es el objetivo a optimizar. Ademas vamos a usar los 5 modelos del CV para estimar el conjunto de test,
# registraremos en optuna las predicciones, matriz de confusion y el score en test.
# CV Score -> Se usa para determinar el rendimiento de los hiperparametros con precision 
# Test Score -> Nos permite testear que esta todo OK, no use (ni debo usar) esos datos para nada en el entrenamiento 
# o la optimizacion de hiperparametros

def cv_es_lgb_objective(trial):

    #PArametros para LightGBM
    lgb_params = {      
                        #PArametros fijos
                        'objective': 'multiclass',
                        'verbosity':-1,
                        'num_class': len(y_train.unique()),
                        #Hiperparametros a optimizar utilizando suggest_float o suggest_int segun el tipo de dato
                        #Se indica el nombre del parametro, valor minimo, valor maximo 
                        #en elgunos casos el parametro log=True para parametros que requieren buscar en esa escala
                        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
                        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),
                        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
                        'feature_fraction': trial.suggest_float('feature_fraction', 0.4, 1.0),
                        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.4, 1.0),
                        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
                        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
                        } 

    #Voy a generar estimaciones de los 5 modelos del CV sobre los datos test y los acumulo en la matriz scores_ensemble
    scores_ensemble = np.zeros((len(y_test),len(y_train.unique())))

    #Score del 5 fold CV inicializado en 0
    score_folds = 0

    #Numero de splits del CV
    n_splits = 5

    #Objeto para hacer el split estratificado de CV
    skf = StratifiedKFold(n_splits=n_splits)

    for i, (if_index, oof_index) in enumerate(skf.split(X_train, y_train)):
        
        #Dataset in fold (donde entreno) 
        lgb_if_dataset = lgb.Dataset(data=X_train.iloc[if_index],
                                        label=y_train.iloc[if_index],
                                        free_raw_data=False)
        
        #Dataset Out of fold (donde mido la performance del CV)
        lgb_oof_dataset = lgb.Dataset(data=X_train.iloc[oof_index],
                                        label=y_train.iloc[oof_index],
                                        free_raw_data=False)

        #Entreno el modelo
        lgb_model = lgb.train(lgb_params,
                                lgb_if_dataset,
                                valid_sets=lgb_oof_dataset,
                                callbacks=[lgb.early_stopping(10, verbose=False)],
                                feval = lgb_custom_metric_kappa
                                )
        
        #Acumulo los scores (probabilidades) de cada clase para cada uno de los modelos que determino en los folds
        #Se predice el 20% de los datos que separe para tes y no uso para entrenar en ningun fold
        scores_ensemble = scores_ensemble + lgb_model.predict(X_test)
        
        #Score del fold (registros de dataset train que en este fold quedan out of fold)
        score_folds = score_folds + cohen_kappa_score(y_train.iloc[oof_index], 
                                                            lgb_model.predict(X_train.iloc[oof_index]).argmax(axis=1),weights = 'quadratic')/n_splits


    #Guardo prediccion del trial sobre el conjunto de test
    # Genero nombre de archivo
    predicted_filename = os.path.join(PATH_TO_TEMP_FILES,f'test_{trial.study.study_name}_{trial.number}.joblib')
    # Copia del dataset para guardar la prediccion
    predicted_df = test.copy()
    # Genero columna pred con predicciones sumadas de los 5 folds
    predicted_df['pred'] = [scores_ensemble[p,:] for p in range(scores_ensemble.shape[0])]
    # Grabo dataframe en temp_artifacts
    dump(predicted_df, predicted_filename)
    # Indico a optuna que asocie el archivo generado al trial
    upload_artifact(trial, predicted_filename, artifact_store)    

    #Grabo natriz de confusion
    #Nombre de archivo
    cm_filename = os.path.join(PATH_TO_TEMP_FILES,f'cm_{trial.study.study_name}_{trial.number}.jpg')
    #Grabo archivo
    plot_confusion_matrix(y_test,scores_ensemble.argmax(axis=1)).write_image(cm_filename)
    #Asocio al trial
    upload_artifact(trial, cm_filename, artifact_store)

    #Determino score en conjunto de test y asocio como metrica adicional en optuna
    test_score = cohen_kappa_score(y_test,scores_ensemble.argmax(axis=1),weights = 'quadratic')
    trial.set_user_attr("test_score", test_score)

    #Devuelvo score del 5fold cv a optuna para que optimice en base a eso
    return(score_folds)

In [68]:
#Inicio el store de artefactos (archivos) de optuna
artifact_store = FileSystemArtifactStore(base_path=PATH_TO_OPTUNA_ARTIFACTS)

#Genero estudio
study = optuna.create_study(direction='maximize',
                            storage="sqlite:///../work/db.sqlite3",  # Specify the storage URL here.
                            study_name="05 - LGB Multiclass CV preproc", #nombre del experimento
                            load_if_exists = True)
#Corro la optimizacion
study.optimize(cv_es_lgb_objective, n_trials=10)

[I 2025-04-15 12:36:54,445] Using an existing study with name '05 - LGB Multiclass CV preproc' instead of creating a new one.
/var/folders/lv/nlwnvhb90677ltl8gcqnky9h0000gp/T/ipykernel_18471/4150719641.py:86: FutureWarning:

upload_artifact() got {'file_path', 'artifact_store', 'study_or_trial'} as positional arguments but they were expected to be given as keyword arguments.

/var/folders/lv/nlwnvhb90677ltl8gcqnky9h0000gp/T/ipykernel_18471/4150719641.py:94: FutureWarning:

upload_artifact() got {'file_path', 'artifact_store', 'study_or_trial'} as positional arguments but they were expected to be given as keyword arguments.

[I 2025-04-15 12:36:58,664] Trial 10 finished with value: 0.35339792303021467 and parameters: {'lambda_l1': 1.1811899841553727e-08, 'lambda_l2': 2.6946356212172535, 'num_leaves': 217, 'feature_fraction': 0.4162679101076012, 'bagging_fraction': 0.7939641247281138, 'bagging_freq': 1, 'min_child_samples': 69}. Best is trial 0 with value: 1.0.
/var/folders/lv/nlwnvhb906

Para ver el optuna dashboard tengo que correr este comando en la terminal

### Desde la terminal
Para cambiar el directorio 
-  cd '/Users/ri1965/Desktop/Maestria DS Austral 2024/11 - Labo 02/work'

Para correr Optuna
- optuna-dashboard sqlite:////Users/ri1965/Desktop/Maestria\ DS\ Austral\ 2024/11\ -\ Labo\ 02/work/db.sqlite3 --artifact-dir /Users/ri1965/Desktop/Maestria\ DS\ Austral\ 2024/11\ -\ Labo\ 02/work/optuna_artifacts --port 8081

Esta era la linea oficial de la cátedra
- #!optuna-dashboard sqlite:///../work/db.sqlite3 --artifact-dir ../work/optuna_artifacts --port 8081

In [69]:
#pip install optuna-dashboard
